In [11]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
import time
from tqdm.notebook import tqdm, trange
import tqdm
from pprint import pprint 
import statsmodels.api as sm
import math

In [12]:
from IPython.display import Image

In [13]:
import folium
from folium import plugins
import importlib

In [14]:
import geemap

In [15]:
import geopandas as gpd
from PIL import Image

In [5]:
#basemaps = util.get_basemaps()

In [6]:
# add_ee_layer = util.add_ee_layer
# folium.Map.add_ee_layer = add_ee_layer

### Start Here!

In [7]:
ee.Authenticate()

KeyboardInterrupt: Interrupted by user

In [16]:
ee.Initialize()

### Run these functions to set up preprocessing

In [17]:
"""
Standard code for adding an EE object to a folium map. copied almost directly from EE Python API 
"""
def add_ee_layer2(self, eeImageObject, visParams, name):
    map_id_dict = ee.Image(eeImageObject).getMapId(visParams)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data &copy; <a href='https://earthengine.google.com/'>Google Earth Engine</a>",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)
folium.Map.add_ee_layer2 = add_ee_layer2

In [21]:
"""
Whenever you visualize something on EE using folium you need to name the band, and as a result,
we have different vis params for whether we are visualizing a single image or a median composite etc 
"""
vis_params_red = {
  'min': 0,
  'max': 6000,
  'palette': ['000000', 'FF0000']}
vis_params_green = {
  'min': 0,
  'max': 6000,
  'palette': ['000000', '00FF00']}
vis_params_blue = {
  'min': 0,
  'max': 6000,
  'palette': ['000000', '0000FF']}
# Set visualization parameters for multiple bands (cannot provide palette)
vis_params_median = {
  'min': 0,
  'max': 4000,
'bands': ['B4_median', 'B3_median', 'B2_median']}
vis_params_median_single = {
  'min': 0,
  'max': 4000,
'bands': ['B12_median']}
vis_params_multi = {
    'min': 0,
  'max': 4000,
'bands': ['B4', 'B3', 'B2']}
vis_params_B = {
    'min': 0,
  'max': 4000,
'bands': ['B']}
vis_params_Bsum = {
    'min': 0,
  'max': 4000,
'bands': ['B_sum']}

In [22]:
# define the polygonal AOI
# This is where we put in Chicago's coordinates
lbx = -87.6298
ubx = lbx + 0.1
lby = 41.8
uby = lby + 0.1


poly = ee.Geometry.Polygon(
    [[[lbx, lby],
      [lbx, uby],
      [ubx, uby],
      [ubx, lby]]])

month = 8
start_year = 2014
end_year = 2018
poi = ee.Geometry.Point(lbx,lby)

Import Sentinel 

In [23]:
col_sent = ee.ImageCollection("COPERNICUS/S2_SR")

filter clouds choose least cloudy

In [62]:
# subset those images with less than 5% cloud cover, then subset those which intersect the 
# lower left corner of the rectangle 'poly' we've just defined
clouds = col_sent.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 
                                      5)).filterBounds(poly)
# can adjust chosen bands to anything you want, and the featurization will flow from it
chosen_bands = ['B4','B3','B2']
# choose only desired bands from the image 
bands = clouds.select(chosen_bands)
s = bands.sort('CLOUD_COVER')

# below, you can choose whether you want to have the least cloudy image "s.first()" or a median of all images in s
# s contains all sentinel images which satisfied the above two filters 

#scene = s.first()

scene = s.reduce(ee.Reducer.median())
chosen_bands = list(map(lambda x: x+'_median',chosen_bands))

In [63]:
fmap = folium.Map(location=[lby, lbx], zoom_start=7)
fmap.add_ee_layer2(scene, vis_params_median, 'Scene 1')
fmap